In [1]:
import numpy as np
import tensorflow as tf
from feedforward import FeedForward
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [2]:
digits = datasets.load_digits()

In [3]:
data = digits.data
targets = digits.target

In [4]:
data.shape

(1797, 64)

In [5]:
targets.shape

(1797,)

In [6]:
set(targets)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [7]:
targets = targets.reshape(-1,1)
data = StandardScaler().fit_transform(data)
idx = np.arange(data.shape[0])
np.random.shuffle(idx)
train_data = data[idx[:1000]]
train_targets = targets[idx[:1000]]
train_targets = LabelBinarizer().fit_transform(train_targets)
validation_data = data[idx[1000:1350]]
validation_targets = targets[idx[1000:1350]]
test_data = data[idx[1350:]]
test_targets = targets[idx[1350:]]

In [8]:
tf.reset_default_graph()
mynnet = FeedForward([data.shape[1], 100, 80, train_targets.shape[1]],
                     activation=tf.nn.relu)
outputs = mynnet.get_output()
labels = mynnet.get_targets()
cost = tf.nn.softmax_cross_entropy_with_logits(outputs, labels)
cost = tf.reduce_mean(cost)
opt = tf.train.AdamOptimizer(learning_rate=0.001)
opt = opt.minimize(cost)

In [9]:
with tf.Session() as sess:
    tf.initialize_all_variables().run(session=sess)
    batch_size = 50
    num_epochs = 100
    num_iter = train_data.shape[0] / batch_size
    train_idx = np.arange(train_data.shape[0])
    train_keep_probs = [0.8, 0.5, 0.5]
    for epoch in xrange(1, num_epochs + 1):
        for i in xrange(num_iter):
            batch_idx = train_idx[i*batch_size: (i*batch_size)+batch_size]
            batch_data = train_data[batch_idx]
            batch_targets = train_targets[batch_idx]

            feed_dict = mynnet.get_train_feed_dict(inputs=batch_data,
                                             targets=batch_targets,
                                             keep_probs=train_keep_probs)
            xentropy, _ = sess.run([cost, opt], feed_dict = feed_dict)
            
            if (i+1) % 10 == 0:
                print 'Epoch: %d | iter: %d | loss: %f'%(epoch, i+1, xentropy)

        if epoch % 10 == 0:
            print '\n       ----- Validation report after epoch %d -------'%epoch
            feed_dict = mynnet.get_test_feed_dict(inputs=validation_data)
            preds = sess.run([outputs], feed_dict=feed_dict)[0]
            preds = tf.nn.softmax(preds).eval(session=sess)
            preds = np.argmax(preds, 1)
            print classification_report(validation_targets, preds)
    
    print '         ------- Test report -------'
    feed_dict = mynnet.get_test_feed_dict(inputs=test_data)
    preds = sess.run([outputs], feed_dict=feed_dict)[0]
    preds = tf.nn.softmax(preds).eval(session=sess)
    preds = np.argmax(preds, 1)
    print classification_report(test_targets, preds)

    print '          ------- Full Data report -------'
    feed_dict = mynnet.get_test_feed_dict(inputs=data)
    preds = sess.run([outputs], feed_dict=feed_dict)[0]
    preds = tf.nn.softmax(preds).eval(session=sess)
    preds = np.argmax(preds, 1)
    print classification_report(targets.reshape(-1, 1), preds)  

Epoch: 1 | iter: 5 | loss: 2.261668
Epoch: 1 | iter: 10 | loss: 2.169602
Epoch: 1 | iter: 15 | loss: 2.002948
Epoch: 1 | iter: 20 | loss: 1.994558
Epoch: 2 | iter: 5 | loss: 1.705863
Epoch: 2 | iter: 10 | loss: 1.776461
Epoch: 2 | iter: 15 | loss: 1.481311
Epoch: 2 | iter: 20 | loss: 1.374791
Epoch: 3 | iter: 5 | loss: 1.247420
Epoch: 3 | iter: 10 | loss: 1.227076
Epoch: 3 | iter: 15 | loss: 0.894372
Epoch: 3 | iter: 20 | loss: 0.878808
Epoch: 4 | iter: 5 | loss: 0.771658
Epoch: 4 | iter: 10 | loss: 0.839722
Epoch: 4 | iter: 15 | loss: 0.597286
Epoch: 4 | iter: 20 | loss: 0.569952
Epoch: 5 | iter: 5 | loss: 0.502674
Epoch: 5 | iter: 10 | loss: 0.677106
Epoch: 5 | iter: 15 | loss: 0.382704
Epoch: 5 | iter: 20 | loss: 0.375047
Epoch: 6 | iter: 5 | loss: 0.409468
Epoch: 6 | iter: 10 | loss: 0.452720
Epoch: 6 | iter: 15 | loss: 0.314033
Epoch: 6 | iter: 20 | loss: 0.283653
Epoch: 7 | iter: 5 | loss: 0.322050
Epoch: 7 | iter: 10 | loss: 0.432296
Epoch: 7 | iter: 15 | loss: 0.310590
Epoch: 7